In [ ]:
!pip install pandas==1.5.3
# !pip install pandas

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import AutoModel

In [ ]:
train_data = pd.read_csv("/kaggle/input/fakenewsdata/Fake_train.csv",encoding='utf8')
train_data.head(10)

In [ ]:
validation_data = pd.read_csv("/kaggle/input/fakenewsdata/Fake_dev.csv",encoding='utf8')

In [ ]:
test_data = pd.read_csv("/kaggle/input/testset/Fake_test_without_labels (1).csv",encoding='utf8')

In [ ]:
train_data['label'].unique()

In [ ]:
label_mapping = {'Fake': 1, 'original': 0}
train_data['label'] = train_data['label'].map(label_mapping)

In [ ]:
train_data.head()

In [ ]:
train_data.to_csv("train_data.csv",index=False)

In [ ]:
train_data.shape

In [ ]:
train_data.isna().sum()

In [ ]:
validation_data.head()

In [ ]:
label_mapping = {'Fake': 1, 'original': 0}
validation_data['label'] = validation_data['label'].map(label_mapping)

In [ ]:
validation_data.head()

In [ ]:
validation_data.shape

In [ ]:
validation_data.head()

In [ ]:
validation_data.to_csv("validation_data.csv",index=False)

In [ ]:
# Concatenate the two DataFrames along rows (axis=0)
merged_data = pd.concat([train_data, validation_data], axis=0, ignore_index=True)

In [ ]:
!pip install git+https://github.com/Hasan-Mesbaul-Ali-Taher/normalizer
from normalizer import normalize
import re

In [ ]:
def preprocess_text(text):

    # Normalize the text using BUET normalizer
    text = normalize(text, unicode_norm="NFKC", punct_replacement= None, url_replacement=" ", emoji_replacement=" ", apply_unicode_norm_last=True)

    # # Define the Bengali Unicode range (Bengali script starts at U+0980)
    # bengali_range = "\u0980-\u09FF"

    # # Remove all characters that are not Bengali words or punctuations
    # processed_text = re.sub(f"[^{bengali_range}!\"#$%&'()*+,-./:;<=>?@[\\]^_`{{|}}~।\s]", "", text)

    # Replace consecutive punctuations with whitespace in between with only the first punctuation
    # processed_text = re.sub(r"([!\"#$%&'()*+,-./:;<=>?@[\\]^_`{{|}}~।])\\1+", r"\1", processed_text)
    processed_text =  re.sub(r'([.,?!|।^_`#@{{|}}~।#$%\&()*+,-./:;<=>?@])\1+', r'\1', text)
    # Split the text into words and remove empty strings caused by consecutive spaces
    words = processed_text.split()
    # Join the words with a single space to form the cleaned text
    text = ' '.join(words)

    result = []
    prev_char = ''

    for char in text:
        if char.strip() == prev_char.strip() and char.strip() in "!।|@#$%^&*()_+-=[]{}|;':\",./<>?.,?!|।^_`#@{{|}}~।#$%\&()*+,-./:;<=>?@":
            continue
        result.append(char)
        prev_char = char

    modified_text = ''.join(result)
    # Define a regular expression to find consecutive punctuations separated by space
    pattern = r'(\s*[\]\[\।\!\@\#\$\%\^\&\*\(\)\_\+\-\=\[\]\{\}\|\;\'\:\"\,\.\/\<\>\?\.\,\?\!\|\।\^\_\`\#\@\{\{\|\}\}\~\।\#\$\%\\\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@]+)+\s*'

    # Use re.sub() to replace consecutive punctuations with a single punctuation
    processed_text = re.sub(pattern, r'\1', modified_text)
    # Remove leading and lagging spaces
    processed_text = processed_text.strip()
    return processed_text

In [ ]:
test_data.shape

In [ ]:
merged_data['text'] = merged_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)

In [ ]:
test_data.head()

In [ ]:
!pip install datasets==2.11.0
# !pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
from datasets import Dataset
# data = load_dataset("csv", data_files="/kaggle/working/train_data.csv")
data = Dataset.from_pandas(merged_data)

In [ ]:
# # validation_data = load_dataset('csv',data_files="/kaggle/working/validation_data.csv")
validation_data = Dataset.from_pandas(validation_data)

In [ ]:
from transformers import AutoTokenizer
model_name = "l3cube-pune/malayalam-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
encoded_text = tokenizer("നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ")
print(encoded_text)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer.model_max_length

In [ ]:
tokenizer.model_input_names

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length",max_length=512, truncation=True)
#     return tokenizer(batch['text'], padding=True,max_length=512, truncation=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_encoded  = data.map(tokenize,batched=True,batch_size=16)

In [ ]:
val_data_encoded  = validation_data.map(tokenize,batched=True,batch_size=16)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_labels=3
model = (AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=num_labels).to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Weighted F1 Score
    f1_weighted = f1_score(labels, preds, average="weighted")
    
    # Macro F1 Score
    f1_macro = f1_score(labels, preds, average="macro")
    
    # Accuracy Score
    acc = accuracy_score(labels, preds)
    
    return {"Accuracy": acc, "Weighted F1 Score": f1_weighted, "Macro F1 Score": f1_macro}


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 16
logging_steps = len(data_encoded)//batch_size
finetuned_model_name = f"{model_name}-FakeNews-Dravidian-finalwithPP"
training_args = TrainingArguments(output_dir = finetuned_model_name,
                                  num_train_epochs=10,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm = False,
                                  logging_steps =logging_steps,
                                  push_to_hub=True,
                                  log_level="error",
                                  )

In [ ]:
from transformers import trainer

trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset = data_encoded,
                  eval_dataset = val_data_encoded,
                  tokenizer = tokenizer,
#                   data_collator=data_collator
                  )

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=0

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
preds_output = trainer.predict(val_data_encoded)
preds_output.metrics

## API Call (BanglaBERT)

In [ ]:
test_data.head()

In [ ]:
from transformers import AutoTokenizer, pipeline
import torch
import pandas as pd

# Load tokenizer and model
model = pipeline("text-classification", model="mdosama39/bert-base-multilingual-cased-FakeNews-Dravidian-finalwithPP", tokenizer="mdosama39/bert-base-multilingual-cased-FakeNews-Dravidian-finalwithPP")
# Load your data
data = test_data
texts = data["text"].tolist()

# Pass the texts to the model for classification
with torch.no_grad():
    predictions = model(texts)

# Get predicted probabilities and labels
predicted_labels = [pred['label'] for pred in predictions]
probs = [pred['score'] for pred in predictions]

# Create new columns in the DataFrame
data['predicted_label'] = predicted_labels
data['predicted_probability'] = probs

In [ ]:
data.head()

In [ ]:
data['label'] = data['predicted_label'].apply(lambda x: 1 if x=='LABEL_1' else (2 if x=='LABEL_2' else 0))

In [ ]:
data.head()

In [ ]:
data = data.drop(['predicted_label',"predicted_probability"],axis=1)

In [ ]:
preds = data['label']

In [ ]:
original_output = pd.read_csv("/kaggle/input/emnlp20230-task1/test_task1.csv",encoding='utf8')
labels = original_output['label']

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
macro_f1 = f1_score(labels, preds, average="macro")

In [ ]:
acc = accuracy_score(labels,preds)
recall = recall_score(labels, preds,average="macro")
precision = precision_score(labels, preds,average="macro")

In [ ]:
macro_f1, acc, precision, recall